Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [109]:
import pandas as pd
import os
import json
import requests as re

In [110]:
with open('yelp_all', 'r') as e: 
    data = json.load(e)
df_yelp_all = pd.DataFrame(data)
df_yelp_all

,coordinates,num_restaurant,rating_x,num_starbucks,rating_y
0,"{'coordinates': '51.52916347,-0.109970527'}",42,2.0,42,2.0
1,"{'coordinates': '51.49960695,-0.197574246'}",18,2.0,18,2.0
2,"{'coordinates': '51.52128377,-0.084605692'}",50,4.5,50,4.5
3,"{'coordinates': '51.50597426,-0.092754157'}",50,3.0,50,3.0
4,"{'coordinates': '51.52395143,-0.122502346'}",50,2.5,50,2.5
...,...,...,...,...,...
791,"{'coordinates': '51.516204,-0.155525'}",50,3.5,50,3.5
792,"{'coordinates': '51.517703,-0.154106'}",50,3.5,50,3.5
793,"{'coordinates': '51.52364804,-0.074754872'}",50,2.5,50,2.5
794,"{'coordinates': '51.49559291,-0.179077626'}",38,2.5,38,2.5


In [ ]:
import re
df_yelp_all[['A','B']] = df_yelp_all['coordinates'].str.split(":", 1, expand=True) 
pattern = r"[{}']"
df_yelp_all["B"] = df_yelp_all["B"].apply(lambda x: re.sub(pattern, '', x))
df_yelp_all = df_yelp_all.drop(['coordinates','A'],axis = 1)


In [ ]:
df_yelp_all[['lat','long']] = df_yelp_all['coordinates'].str.split(",", 1, expand=True) 
df_yelp_all

In [115]:
df_yelp_all = df_yelp_all.rename(columns={'B':'coordinates'})
df_yelp_all

,num_restaurant,rating_x,num_starbucks,rating_y,coordinates
0,42,2.0,42,2.0,"51.52916347,-0.109970527"
1,18,2.0,18,2.0,"51.49960695,-0.197574246"
2,50,4.5,50,4.5,"51.52128377,-0.084605692"
3,50,3.0,50,3.0,"51.50597426,-0.092754157"
4,50,2.5,50,2.5,"51.52395143,-0.122502346"
...,...,...,...,...,...
791,50,3.5,50,3.5,"51.516204,-0.155525"
792,50,3.5,50,3.5,"51.517703,-0.154106"
793,50,2.5,50,2.5,"51.52364804,-0.074754872"
794,38,2.5,38,2.5,"51.49559291,-0.179077626"


In [116]:
with open('citybike', 'r') as g: 
    data = json.load(g)
df_all_stations = pd.DataFrame(data)

In [117]:
df_all_stations[['lat','long']] = df_all_stations['coordinates'].str.split(",", 1, expand=True)
df_all_stations

,name,coordinates,bikes,lat,long
0,"001023 - River Street , Clerkenwell","51.52916347,-0.109970527",12,51.52916347,-0.109970527
1,"001018 - Phillimore Gardens, Kensington","51.49960695,-0.197574246",31,51.49960695,-0.197574246
2,"001012 - Christopher Street, Liverpool Street","51.52128377,-0.084605692",4,51.52128377,-0.084605692
3,"001024 - Park Street, Bankside","51.50597426,-0.092754157",10,51.50597426,-0.092754157
4,"001022 - Brunswick Square, Bloomsbury","51.52395143,-0.122502346",16,51.52395143,-0.122502346
...,...,...,...,...,...
794,"200008 - Portman Square, Marylebone","51.516204,-0.155525",3,51.516204,-0.155525
795,"001225 - George Street, Marylebone","51.517703,-0.154106",25,51.517703,-0.154106
796,"000973 - Bethnal Green Road, Shoreditch","51.52364804,-0.074754872",20,51.52364804,-0.074754872
797,"002677 - Natural History Museum, South Kensington","51.49559291,-0.179077626",9,51.49559291,-0.179077626


In [ ]:
df_all_stations['lat'] = df_all_stations['lat'].str.strip()
df_all_stations['long'] = df_all_stations['long'].str.strip()
df_yelp_all['lat'] = df_yelp_all['lat'].str.strip()
df_yelp_all['long'] = df_yelp_all['long'].str.strip()

In [ ]:
df_final = pd.merge(df_yelp_all, df_all_stations, on=['lat','long'],how='left')
df_final = df_final.drop(['coordinates_x','coordinates_y'],axis = 1)
df_final

df_final.to_json('df_final', orient = 'records')

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:

df_final.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (7,7))
ax= sns.heatmap(df_final.corr(), annot = True, cmap="YlGn",linewidth =1)

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [ ]:
connection = create_connection("E:\LHL\WEEK3\Statistical-Modelling-Project\df_final.sqlite")
conn = sqlite3.connect("E:\LHL\WEEK3\Statistical-Modelling-Project\df_final.sqlite")
df_final.to_sql("df_final_data", conn, if_exists='replace')


Look at the data before and after the join to validate your data.

In [ ]:
df_all = pd.read_sql("""SELECT * FROM df_final_data """, conn)
df_all